<a href="https://colab.research.google.com/github/Ixiondbz/CSE499/blob/main/RCNN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!pip install tensorflow==2.7.0
!pip install -U plotly

import json
from pathlib import Path
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm import tqdm
import pandas as pd
import random
import cv2

import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import math
import cv2
import copy
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import os

from sklearn.metrics import average_precision_score

from keras import backend as K
from tensorflow.keras.optimizers import SGD, RMSprop
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.utils.layer_utils import get_source_inputs
#from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from tensorflow.keras.metrics import categorical_crossentropy

from keras.models import Model
from keras.utils import generic_utils
#tf.keras.layers.InputSpec
from tensorflow.keras.layers import Layer, InputSpec
from keras import initializers, regularizers

     |████████████████████████████████| 25.3 MB 1.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [3]:
from google.colab import drive
drive.mount('/drive')

folder = "MyDrive/MIDOG_Challenge" #@param {type:"string"}
midog_folder = Path("/drive") / Path(folder)

print(list(midog_folder.glob("*.*")))

Mounted at /drive
[PosixPath('/drive/MyDrive/MIDOG_Challenge/MIDOG.sqlite'), PosixPath('/drive/MyDrive/MIDOG_Challenge/MIDOG.json')]


In [4]:
!apt-get install python3-openslide
from openslide import open_slide

Selecting previously unselected package javascript-common.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../00-javascript-common_11_all.deb ...
Unpacking javascript-common (11) ...
Selecting previously unselected package libjs-jquery.
Preparing to unpack .../01-libjs-jquery_3.2.1-1_all.deb ...
Unpacking libjs-jquery (3.2.1-1) ...
Selecting previously unselected package libopenslide0.
Preparing to unpack .../02-libopenslide0_3.4.1+dfsg-2_amd64.deb ...
Unpacking libopenslide0 (3.4.1+dfsg-2) ...
Selecting previously unselected package python-asn1crypto.
Preparing to unpack .../03-python-asn1crypto_0.24.0-1_all.deb ...
Unpacking python-asn1crypto (0.24.0-1) ...
Selecting previously unselected package python-blinker.
Preparing to unpack .../04-python-blinker_1.4+dfsg1-0.1_all.deb ...
Unpacking python-blinker (1.4+dfsg1-0.1) ...
Selecting previously unselected package python-cffi-backend.
Preparing to unpack .../05-python-cffi-backend_1.11.5-1_

In [5]:
image_folder = midog_folder / "images"

hamamatsu_rx_ids = list(range(0, 51))
hamamatsu_360_ids = list(range(51, 101))
aperio_ids = list(range(101, 151))
leica_ids = list(range(151, 201))


annotation_file = midog_folder / "MIDOG.json"
rows = []
with open(annotation_file) as f:
    data = json.load(f)

    categories = {1: 'mitotic figure', 2: 'hard negative'}

    for row in data["images"]:
        file_name = row["file_name"]
        image_id = row["id"]
        width = row["width"]
        height = row["height"]

        scanner  = "Hamamatsu XR"
        if image_id in hamamatsu_360_ids:
            scanner  = "Hamamatsu S360"
        if image_id in aperio_ids:
            scanner  = "Aperio CS"
        if image_id in leica_ids:
            scanner  = "Leica GT450"
         
        for annotation in [anno for anno in data['annotations'] if anno["image_id"] == image_id]:
            box = annotation["bbox"]
            cat = categories[annotation["category_id"]]

            rows.append([file_name, image_id, width, height, box, cat, scanner])

df = pd.DataFrame(rows, columns=["file_name", "image_id", "width", "height", "box", "cat", "scanner"])

In [6]:
train = df

In [7]:
train.head()

,file_name,image_id,width,height,box,cat,scanner
0,001.tiff,1,7215,5412,"[4336, 346, 4386, 396]",hard negative,Hamamatsu XR
1,001.tiff,1,7215,5412,"[756, 872, 806, 922]",hard negative,Hamamatsu XR
2,001.tiff,1,7215,5412,"[270, 4044, 320, 4094]",hard negative,Hamamatsu XR
3,001.tiff,1,7215,5412,"[6672.5, 706.5, 6722.5, 756.5]",hard negative,Hamamatsu XR
4,002.tiff,2,7215,5412,"[1872, 319, 1922, 369]",hard negative,Hamamatsu XR


In [26]:
train[train['file_name']=='001.tiff']

,file_name,image_id,width,height,box,cat,scanner
0,001.tiff,1,7215,5412,"[4336, 346, 4386, 396]",hard negative,Hamamatsu XR
1,001.tiff,1,7215,5412,"[756, 872, 806, 922]",hard negative,Hamamatsu XR
2,001.tiff,1,7215,5412,"[270, 4044, 320, 4094]",hard negative,Hamamatsu XR
3,001.tiff,1,7215,5412,"[6672.5, 706.5, 6722.5, 756.5]",hard negative,Hamamatsu XR


In [8]:
train['cat'].unique()

array(['hard negative', 'mitotic figure'], dtype=object)

In [9]:
train.dtypes

file_name    object
image_id      int64
width         int64
height        int64
box          object
cat          object
scanner      object
dtype: object

In [10]:
train['box']

0               [4336, 346, 4386, 396]
1                 [756, 872, 806, 922]
2               [270, 4044, 320, 4094]
3       [6672.5, 706.5, 6722.5, 756.5]
4               [1872, 319, 1922, 369]
                     ...              
4430            [569, 4276, 619, 4326]
4431          [3730, 4538, 3780, 4588]
4432          [4318, 4138, 4368, 4188]
4433          [5643, 1318, 5693, 1368]
4434      [2681.5, 2846, 2731.5, 2896]
Name: box, Length: 4435, dtype: object

In [11]:
box_dataframe = train.copy(deep=True)

In [12]:
box_dataframe.drop(['file_name','width','height','cat','scanner'], axis=1, inplace=True)

In [13]:
box_dataframe.head()

,image_id,box
0,1,"[4336, 346, 4386, 396]"
1,1,"[756, 872, 806, 922]"
2,1,"[270, 4044, 320, 4094]"
3,1,"[6672.5, 706.5, 6722.5, 756.5]"
4,2,"[1872, 319, 1922, 369]"


In [14]:
size = box_dataframe.shape[0]

In [ ]:
List = box_dataframe[box_dataframe['image_id']==57]['box']
List 

In [15]:
for i in range(size):
  FileName = str(i+1)+".tiff"
  File = open("sample.txt", "w")
   
integer = 10
File.write(str(integer))
File.close()